In [4]:
# 필요한 라이브러리 설치
!pip install torch
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.8.0+cu101.html
!pip install torch_geometric
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install imbalanced-learn
!pip install optuna

# 필요한 라이브러리 임포트
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.nn import GATConv
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.over_sampling import SMOTE
import optuna

# CUDA 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Looking in links: https://data.pyg.org/whl/torch-1.8.0+cu101.html


In [56]:
# 데이터 파일 경로 설정
train_files = [
    r'C:\Users\Researcher\Desktop\MTAD-GAT\train\train1.csv',
    r'C:\Users\Researcher\Desktop\MTAD-GAT\train\train2.csv'
]
test_files = [
    r'C:\Users\Researcher\Desktop\MTAD-GAT\test1\test1.csv',
    r'C:\Users\Researcher\Desktop\MTAD-GAT\test1\test2.csv'
]

# 데이터 로딩 함수
def load_data(files):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        df.drop(columns=['time'], inplace=True)  # 'time' 열 제거
        df_list.append(df)
    combined_df = pd.concat(df_list, axis=0, ignore_index=True)
    return combined_df

# prepare_data 함수 정의
def prepare_data(df, label_column):
    inputs = torch.tensor(df.drop(columns=[label_column]).values, dtype=torch.float32)
    labels = torch.tensor(df[label_column].values, dtype=torch.long)  # 라벨을 정수형으로 변환
    return inputs, labels

# edge_index 생성 함수
def create_edge_index(num_nodes):
    edge_index = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                edge_index.append([i, j])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return edge_index

# 데이터 전처리 함수
def preprocess_data(train_files, test_files, label_column='attack'):
    print("Loading data...")
    train_df = load_data(train_files)
    test_df = load_data(test_files)
    print("Data loaded.")

    scaler = MinMaxScaler()  # MinMaxScaler를 사용하여 데이터 스케일링
    train_scaled = scaler.fit_transform(train_df.drop(columns=[label_column]))
    test_scaled = scaler.transform(test_df.drop(columns=[label_column]))

    train_scaled_df = pd.DataFrame(train_scaled, columns=train_df.columns.drop(label_column))
    test_scaled_df = pd.DataFrame(test_scaled, columns=test_df.columns.drop(label_column))

    train_scaled_df[label_column] = train_df[label_column].values
    test_scaled_df[label_column] = test_df[label_column].values

    train_inputs, train_labels = prepare_data(train_scaled_df, label_column)
    test_inputs, test_labels = prepare_data(test_scaled_df, label_column)

    smote = SMOTE(random_state=42)
    train_inputs_smote, train_labels_smote = smote.fit_resample(train_inputs.cpu().numpy(), train_labels.cpu().numpy())
    print("SMOTE applied.")

    print(f"train_inputs_smote shape: {train_inputs_smote.shape}, dtype: {train_inputs_smote.dtype}")
    print(f"train_labels_smote shape: {train_labels_smote.shape}, dtype: {train_labels_smote.dtype}")
    
    # 데이터 값 범위 확인
    print(f"train_inputs_smote min: {np.min(train_inputs_smote)}, max: {np.max(train_inputs_smote)}")
    print(f"train_labels_smote min: {np.min(train_labels_smote)}, max: {np.max(train_labels_smote)}")

    train_inputs_smote = torch.tensor(train_inputs_smote, dtype=torch.float32).to(device)
    train_labels_smote = torch.tensor(train_labels_smote, dtype=torch.long).to(device)  # 라벨을 정수형으로 변환

    print(f"train_inputs_smote shape: {train_inputs_smote.shape}, dtype: {train_inputs_smote.dtype}")
    print(f"train_labels_smote shape: {train_labels_smote.shape}, dtype: {train_labels_smote.dtype}")

    train_dataset_smote = TensorDataset(train_inputs_smote, train_labels_smote)
    train_loader_smote = DataLoader(train_dataset_smote, batch_size=32, shuffle=True)

    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
    test_dataset = TensorDataset(test_inputs, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    num_nodes = train_inputs_smote.size(1)
    edge_index = create_edge_index(num_nodes)

    print("Data preprocessed.")
    return train_loader_smote, test_loader, train_df, test_df, edge_index

# 데이터 전처리 호출
train_loader_smote, test_loader, train_df, test_df, edge_index = preprocess_data(train_files, test_files)
print(f"Train Loader Size: {len(train_loader_smote)}")
print(f"Test Loader Size: {len(test_loader)}")
print(f"Edge Index: {edge_index.shape}")

Loading data...
Data loaded.
SMOTE applied.
train_inputs_smote shape: (1100048, 62), dtype: float32
train_labels_smote shape: (1100048,), dtype: int64
train_inputs_smote min: 0.0, max: 1.0
train_labels_smote min: 0, max: 1


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [54]:

# 모델 정의
class MTADGAT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super(MTADGAT, self).__init__()
        self.gat1_1 = GATConv(input_dim, hidden_dim, heads=num_heads, concat=True)
        self.gat1_2 = GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=True)
        self.gat1_3 = GATConv(hidden_dim, output_dim, heads=1, concat=True)
        self.gat2_1 = GATConv(input_dim, hidden_dim, heads=num_heads, concat=True)
        self.gat2_2 = GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=True)
        self.gat2_3 = GATConv(hidden_dim, output_dim, heads=1, concat=True)
        self.gru = nn.GRU(output_dim * 2, hidden_dim, batch_first=True)
        self.fc_forecasting = nn.Linear(hidden_dim, input_dim)
        self.fc_reconstruction = nn.Linear(hidden_dim, input_dim)

    def forward(self, x, edge_index1, edge_index2):
        x1 = self.gat1_1(x, edge_index1)
        x1 = torch.relu(x1)
        x1 = self.gat1_2(x1, edge_index1)
        x1 = torch.relu(x1)
        x1 = self.gat1_3(x1, edge_index1)

        x2 = self.gat2_1(x, edge_index2)
        x2 = torch.relu(x2)
        x2 = self.gat2_2(x2, edge_index2)
        x2 = torch.relu(x2)
        x2 = self.gat2_3(x2, edge_index2)

        x = torch.cat([x1, x2], dim=1)
        x = x.view(1, -1, x.size(1))
        x, _ = self.gru(x)
        x = x.view(-1, x.size(2))

        x_forecasting = self.fc_forecasting(x)
        x_reconstruction = self.fc_reconstruction(x)

        return x_forecasting, x_reconstruction

# 손실 함수 및 옵티마이저 정의
def get_class_weights(train_df, num_classes):
    class_counts = train_df['attack'].value_counts().sort_index()
    class_weights = torch.ones(num_classes, dtype=torch.float32).to(device)
    for i in range(len(class_counts)):
        class_weights[i] = class_counts.max() / class_counts.iloc[i]
    return class_weights

def get_criteria_and_optimizer(model, class_weights, learning_rate):
    criterion_forecasting = nn.CrossEntropyLoss(weight=class_weights)
    criterion_reconstruction = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return criterion_forecasting, criterion_reconstruction, optimizer


# 첫 번째 배치 확인
print("Checking first batch of train loader:")
for inputs, labels in train_loader_smote:
    print(f"Inputs shape: {inputs.shape}, Labels shape: {labels.shape}")
    break
    
# 모델 점검
input_dim = train_df.shape[1] - 1
hidden_dim = 64
output_dim = 32
num_heads = 8
model = MTADGAT(input_dim, hidden_dim, output_dim, num_heads).to(device)
print(f"Model initialized with input_dim={input_dim}, hidden_dim={hidden_dim}, output_dim={output_dim}, num_heads={num_heads}")

# 모델 테스트 함수
def test_model(model, inputs, edge_index1, edge_index2):
    model.eval()
    with torch.no_grad():
        try:
            print("Testing forward pass...")
            outputs_forecasting, outputs_reconstruction = model(inputs, edge_index1.to(device), edge_index2.to(device))
            print(f"Outputs Forecasting shape: {outputs_forecasting.shape}, Outputs Reconstruction shape: {outputs_reconstruction.shape}")
            print("Forward pass successful.")
        except Exception as e:
            print(f"Error during model testing: {e}")

# 첫 번째 배치를 사용하여 모델 테스트
print("Testing model with first batch...")
inputs, labels = next(iter(train_loader_smote))
test_model(model, inputs, edge_index, edge_index)

Checking first batch of train loader:


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [46]:
def train_and_evaluate(model, train_loader_smote, test_loader, criterion_forecasting, criterion_reconstruction, optimizer, edge_index1, edge_index2, patience=10):
    epochs = 50 
    train_losses = []
    test_losses = []
    precisions = []
    recalls = []
    roc_aucs = []
    accuracies = []

    best_roc_auc = 0
    best_epoch = 0
    epochs_no_improve = 0

    print("Starting training loop...")

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        print(f"Epoch {epoch+1}/{epochs} 시작...")

        for i, (inputs, labels) in enumerate(train_loader_smote):
            print(f"Batch {i+1}/{len(train_loader_smote)} 시작...")
            print(f"Inputs shape: {inputs.shape}, Labels shape: {labels.shape}")
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            try:
                print("Forward pass 시작...")
                outputs_forecasting, outputs_reconstruction = model(inputs, edge_index1.to(device), edge_index2.to(device))
                print(f"Outputs Forecasting shape: {outputs_forecasting.shape}, Outputs Reconstruction shape: {outputs_reconstruction.shape}")
                print("Forward pass 완료")
                
                print("Loss 계산 시작...")
                loss_forecasting = criterion_forecasting(outputs_forecasting, labels.long())
                loss_reconstruction = criterion_reconstruction(outputs_reconstruction, inputs)
                loss = loss_forecasting + loss_reconstruction
                print(f"Loss 계산 완료: {loss.item():.4f}")
                
                print("Backward pass 시작...")
                loss.backward()
                optimizer.step()
                print("Backward pass 완료")
                
                total_loss += loss.item()
                print(f"Batch {i+1}/{len(train_loader_smote)} 완료, Loss: {loss.item():.4f}")
            except Exception as e:
                print(f"Error during training at epoch {epoch}, batch {i}: {e}")
                return

        avg_train_loss = total_loss / len(train_loader_smote)
        train_losses.append(avg_train_loss)

        print(f"Epoch {epoch+1}/{epochs} Train Loss: {avg_train_loss:.4f}")

        model.eval()
        all_preds = []
        all_labels = []
        total_test_loss = 0
        predictions = []
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(test_loader):
                print(f"Test Batch {i+1}/{len(test_loader)} 시작...")
                inputs, labels = inputs.to(device), labels.to(device)
                try:
                    outputs_forecasting, outputs_reconstruction = model(inputs, edge_index1.to(device), edge_index2.to(device))
                    print(f"Outputs Forecasting shape: {outputs_forecasting.shape}, Outputs Reconstruction shape: {outputs_reconstruction.shape}")
                    loss_forecasting = criterion_forecasting(outputs_forecasting, labels.long())
                    loss_reconstruction = criterion_reconstruction(outputs_reconstruction, inputs)
                    loss = loss_forecasting + loss_reconstruction
                    total_test_loss += loss.item()
                    preds = outputs_forecasting.argmax(dim=1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                    predictions.extend(outputs_forecasting.cpu().numpy().flatten())
                    print(f"Test Batch {i+1}/{len(test_loader)} 완료, Loss: {loss.item():.4f}")
                except Exception as e:
                    print(f"Error during evaluation at epoch {epoch}, batch {i}: {e}")
                    return

        avg_test_loss = total_test_loss / len(test_loader)
        test_losses.append(avg_test_loss)

        print(f"Epoch {epoch+1}/{epochs} Test Loss: {avg_test_loss:.4f}")

        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)
        predictions = np.array(predictions)

        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        precisions.append(precision)
        recalls.append(recall)

        accuracy = accuracy_score(all_labels, all_preds)
        accuracies.append(accuracy)

        try:
            fpr, tpr, _ = roc_curve(all_labels, predictions)
            roc_auc = auc(fpr, tpr)
            roc_aucs.append(roc_auc)
        except ValueError as e:
            print(f"ROC AUC calculation error at epoch {epoch}: {e}")
            roc_auc = 0.0
            roc_aucs.append(roc_auc)

        print(f"Epoch {epoch+1}/{epochs} Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, ROC AUC: {roc_auc:.4f}")

        if roc_auc > best_roc_auc:
            best_roc_auc = roc_auc
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pth')
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    print("Loading best model for evaluation...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_labels = np.array(all_labels)
    predictions = np.array(predictions)
    evaluate_model(torch.tensor(test_labels), torch.tensor(predictions), train_losses, test_losses, precisions, recalls, accuracies)

    print("Training and evaluation completed.")=

SyntaxError: cannot assign to function call (1875901876.py, line 125)

In [59]:
def evaluate_model(test_labels, predictions, train_losses, test_losses, precisions, recalls, accuracies):
    mse = np.mean((test_labels.numpy() - predictions) ** 2)
    mae = mean_absolute_error(test_labels.numpy(), predictions)
    r2 = r2_score(test_labels.numpy(), predictions)

    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"R^2 Score: {r2}")

    anomaly_threshold = mse + 3 * np.std(predictions - test_labels.numpy().reshape(-1, 1))
    anomalies = (np.abs(predictions - test_labels.numpy().reshape(-1, 1)) > anomaly_threshold).astype(int)

    accuracy = accuracy_score(test_labels.numpy(), anomalies)
    f1 = f1_score(test_labels.numpy(), anomalies, average='macro')
    cm = confusion_matrix(test_labels.numpy(), anomalies)

    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print("Confusion Matrix:")
    print(cm)

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()

    plt.figure(figsize=(15, 5))
    plt.plot(test_labels.numpy(), label='Actual', color='blue')
    plt.plot(predictions, label='Predicted', color='green')
    plt.scatter(np.where(anomalies == 1)[0], predictions[anomalies == 1], color='red', label='Anomalies')
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Testing Loss')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(precisions, label='Precision')
    plt.plot(recalls, label='Recall')
    plt.plot(accuracies, label='Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.title('Precision, Recall, and Accuracy')
    plt.show()

    fpr, tpr, _ = roc_curve(test_labels.numpy(), predictions)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

In [47]:
# 데이터 전처리 함수
def preprocess_data(train_files, test_files, label_column='attack'):
    train_df = load_data(train_files)
    test_df = load_data(test_files)

    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_df.drop(columns=[label_column]))
    test_scaled = scaler.transform(test_df.drop(columns=[label_column]))

    train_scaled_df = pd.DataFrame(train_scaled, columns=train_df.columns.drop(label_column))
    test_scaled_df = pd.DataFrame(test_scaled, columns=test_df.columns.drop(label_column))

    train_scaled_df[label_column] = train_df[label_column].values
    test_scaled_df[label_column] = test_df[label_column].values

    train_inputs, train_labels = prepare_data(train_scaled_df, label_column)
    test_inputs, test_labels = prepare_data(test_scaled_df, label_column)

    smote = SMOTE(random_state=42)
    train_inputs_smote, train_labels_smote = smote.fit_resample(train_inputs.cpu(), train_labels.cpu())
    train_inputs_smote = torch.tensor(train_inputs_smote, dtype=torch.float32).to(device)
    train_labels_smote = torch.tensor(train_labels_smote, dtype=torch.float32).to(device)

    train_dataset_smote = TensorDataset(train_inputs_smote, train_labels_smote)
    train_loader_smote = DataLoader(train_dataset_smote, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(test_inputs, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    num_nodes = train_inputs_smote.size(1)
    edge_index = create_edge_index(num_nodes)

    return train_loader_smote, test_loader, train_df, test_df, edge_index

# objective 함수
def objective(trial):
    train_files = [
            r'C:\Users\Researcher\Desktop\MTAD-GAT\train\train1.csv',
            r'C:\Users\Researcher\Desktop\MTAD-GAT\train\train2.csv'
    ]
    test_files = [
         r'C:\Users\Researcher\Desktop\MTAD-GAT\test1\test1.csv',
         r'C:\Users\Researcher\Desktop\MTAD-GAT\test1\test2.csv'
    ]


    # 하이퍼파라미터 샘플링
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    output_dim = trial.suggest_int('output_dim', 64, 256)
    num_heads = trial.suggest_int('num_heads', 1, 8)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    epochs = trial.suggest_int('epochs', 10, 100)

    # 데이터 전처리
    train_loader_smote, test_loader, train_df, test_df, edge_index = preprocess_data(train_files, test_files)

    # 모델 초기화
    num_nodes = train_loader_smote.dataset.tensors[0].shape[1]
    model = MTADGAT(input_dim=num_nodes, hidden_dim=hidden_dim, output_dim=output_dim, num_heads=num_heads).to(device)

    # 손실 함수 및 옵티마이저 설정
    num_classes = train_df['attack'].nunique()
    class_weights = get_class_weights(train_df, num_classes)
    criterion_forecasting, criterion_reconstruction, optimizer = get_criteria_and_optimizer(model, class_weights, learning_rate)

    # 모델 학습 및 평가
    try:
        train_losses, test_losses, precisions, recalls, accuracies, roc_aucs = train_and_evaluate(
            model, train_loader_smote, test_loader, criterion_forecasting, criterion_reconstruction, optimizer, edge_index, edge_index, epochs)
    except Exception as e:
        print(f"Trial failed due to error: {e}")
        return float('nan')

    return max(roc_aucs)  # ROC AUC 최대화

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 트라이얼이 완료되었는지 확인
if len(study.trials) == 0 or all([t.state != optuna.trial.TrialState.COMPLETE for t in study.trials]):
    raise ValueError("No trials are completed yet.")

print('Best trial:')
trial = study.best_trial
print(' Value: ', trial.value)
print(' Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

In [48]:

def main():
    # 업로드된 파일 경로 설정
    train_files = [
        '/content/train/train1.csv',
        '/content/train/train2.csv'
    ]
    test_files = [
        '/content/test/test1.csv',
        '/content/test/test2.csv'
    ]

    best_params = study.best_params
    print(f"Optimal number of epochs: {best_params['epochs']}")  # Print the optimal number of epochs

    train_loader_smote, test_loader, train_df, test_df = preprocess_data(train_files, test_files)

    # edge_index 생성
    edge_index1 = create_edge_index(train_df)
    edge_index2 = create_edge_index(test_df)

    num_nodes = train_loader_smote.dataset.tensors[0].shape[1]
    model = MTADGAT(input_dim=num_nodes, hidden_dim=best_params['hidden_dim'], output_dim=best_params['output_dim'], num_heads=best_params['num_heads']).to(device)

    num_classes = train_df['attack'].nunique()
    class_weights = get_class_weights(train_df, num_classes)
    criterion_forecasting, criterion_reconstruction, optimizer = get_criteria_and_optimizer(model, class_weights, best_params['learning_rate'])

    train_losses, test_losses, precisions, recalls, accuracies, roc_aucs = train_and_evaluate(
        model, train_loader_smote, test_loader, criterion_forecasting, criterion_reconstruction, optimizer, edge_index1, edge_index2, best_params['epochs'])

    test_inputs, test_labels = next(iter(test_loader))
    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
    model.eval()
    with torch.no_grad():
        outputs_forecasting, _ = model(test_inputs, edge_index1.to(device), edge_index2.to(device))
        predictions = outputs_forecasting.argmax(dim=1).cpu().numpy()

    evaluate_model(test_labels, predictions, train_losses, test_losses, precisions, recalls, accuracies)

if __name__ == "__main__":
    main()

[I 2024-06-13 06:17:50,594] A new study created in memory with name: no-name-c099f7ca-b192-4d1f-bca9-0b8c838f1f41


Loading data...
Data loaded.


[W 2024-06-13 06:17:55,419] Trial 0 failed with parameters: {'hidden_dim': 64, 'output_dim': 111, 'num_heads': 3, 'learning_rate': 0.000418502895230389, 'epochs': 87} because of the following error: RuntimeError('CUDA error: device-side assert triggered\nCompile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.\n').
Traceback (most recent call last):
  File "C:\Users\Researcher\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Researcher\AppData\Local\Temp\ipykernel_6664\3911277233.py", line 19, in objective
    train_loader_smote, test_loader, train_df, test_df, edge_index = preprocess_data(train_files, test_files)
                                                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Researcher\AppData\Local\Temp\ipykernel_6664\2038594078.py", line 59, in preprocess_data
    train_inputs_smote = torch.tens

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [49]:
# 최적 하이퍼파라미터로 모델 학습 및 평가
best_params = study.best_params
print(f"Optimal number of epochs: {best_params['epochs']}")

train_loader_smote, test_loader = preprocess_data(train_df, test_df)
edge_index1 = create_edge_index(train_df)
edge_index2 = create_edge_index(train_df)
num_nodes = train_loader_smote.dataset.tensors[0].shape[1]
model = MTADGAT(input_dim=num_nodes, hidden_dim=best_params['hidden_dim'], output_dim=best_params['output_dim'], num_heads=best_params['num_heads']).to(device)

num_classes = train_df['attack'].nunique()
class_weights = get_class_weights(train_df, num_classes)
criterion_forecasting, criterion_reconstruction, optimizer = get_criteria_and_optimizer(model, class_weights, best_params['learning_rate'])

train_losses, test_losses, precisions, recalls, accuracies, roc_aucs = train_and_evaluate(
    model, train_loader_smote, test_loader, criterion_forecasting, criterion_reconstruction, optimizer, edge_index1, edge_index2, best_params['epochs'])

print("Model training and evaluation with optimal hyperparameters completed.")

ValueError: No trials are completed yet.